In [ ]:
import psycopg2
import tkinter as tk
from tkinter import messagebox, simpledialog, Toplevel, Label

DB_CONFIG = {
    'dbname': 'fires',
    'user': 'postgres',
    'password': '----',
    'host': 'localhost',
    'port': '5432'
}

# Connect to the PostgreSQL database
def connect_db():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except psycopg2.OperationalError as e:
        messagebox.showerror("Database Error", f"Could not connect to the database:\n{e}")
        return None

# Search fire incidents by locality
def search_fire_incident():
    conn = connect_db()
    if conn is None:
        return

    locality = simpledialog.askstring("Input", "Enter the locality to search:")
    if locality:
        cursor = conn.cursor()
        query =
        cursor.execute(query, (f"%{locality}%",))
        results = cursor.fetchall()

        if results:
            incidents_text = "\n".join(
                f"{result[0]} | {result[1]} ha | {result[2]} hours" for result in results
            )
            messagebox.showinfo(f"Fire incidents in {locality}", incidents_text)
        else:
            messagebox.showinfo("No Results", f"No incidents found in {locality}.")

        cursor.close()
    else:
        messagebox.showwarning("Input Needed", "Please enter a locality to search.")
    conn.close()

# Search firefighter by ID or Name
def search_firefighter():
    conn = connect_db()
    if conn is None:
        return

    firefighter_id = simpledialog.askstring("Input", "Enter Firefighter ID or Name:")
    if firefighter_id:
        cursor = conn.cursor()
        query = """
        SELECT * FROM Bombeiros
        WHERE CAST(BombeiroID AS TEXT) = %s OR Nome ILIKE %s;
        """
        cursor.execute(query, (firefighter_id, f"%{firefighter_id}%"))
        results = cursor.fetchall()

        if results:
            firefighter_text = "\n".join(
                f"ID: {result[0]}, Name: {result[1]}, DOB: {result[2]}, Admission Date: {result[3]}, Position: {result[4]}, QuartelID: {result[5]}"
                for result in results
            )
            messagebox.showinfo("Firefighter Information", firefighter_text)
        else:
            messagebox.showinfo("No Results", f"No firefighter found with ID or name '{firefighter_id}'.")

        cursor.close()
    else:
        messagebox.showwarning("Input Needed", "Please enter a Firefighter ID or Name.")
    conn.close()

# Add or Update Vehicle Information
def add_update_vehicle():
    conn = connect_db()
    if conn is None:
        return

    vehicle_id = simpledialog.askstring("Input", "Enter Vehicle ID (or leave blank to add new vehicle):")
    placa = simpledialog.askstring("Input", "Enter Vehicle Plate:")
    year = simpledialog.askinteger("Input", "Enter Vehicle Year:")
    model = simpledialog.askstring("Input", "Enter Vehicle Model:")
    quartel_id = simpledialog.askinteger("Input", "Enter Quartel ID:")

    cursor = conn.cursor()
    if vehicle_id:
        # Update existing vehicle
        query = """
        UPDATE Viaturas
        SET Placa = %s, Ano = %s, Modelo = %s, QuartelID = %s
        WHERE ViaturaID = %s;
        """
        cursor.execute(query, (placa, year, model, quartel_id, vehicle_id))
        conn.commit()
        messagebox.showinfo("Vehicle Update", f"Vehicle with ID {vehicle_id} has been updated.")
    else:
        # Add a new vehicle
        query = """
        INSERT INTO Viaturas (Placa, Ano, Modelo, QuartelID)
        VALUES (%s, %s, %s, %s)
        RETURNING ViaturaID;
        """
        cursor.execute(query, (placa, year, model, quartel_id))
        new_id = cursor.fetchone()[0]
        conn.commit()
        messagebox.showinfo("Vehicle Added", f"New vehicle added with ID {new_id}.")

    cursor.close()
    conn.close()

# Show top fire incidents by area
def show_top_incidents_by_area():
    conn = connect_db()
    if conn is None:
        return

    n = simpledialog.askinteger("Input", "How many top incidents would you like to see?", initialvalue=5)
    cursor = conn.cursor()
    query = """
    SELECT DataHoraAlerta, Localizacao.Local, AreaTotal_ha, Duracao_Horas
    FROM RegistroIncendio
    JOIN Localizacao ON RegistroIncendio.Codigo_SGIF = Localizacao.Codigo_SGIF
    ORDER BY AreaTotal_ha DESC
    LIMIT %s;
    """
    cursor.execute(query, (n,))
    results = cursor.fetchall()

    if results:
        incidents_text = "\n".join(
            f"{result[0]} | {result[1]} | {result[2]} ha | {result[3]} hours" for result in results
        )
        messagebox.showinfo("Top Fire Incidents by Area", incidents_text)
    else:
        messagebox.showinfo("No Results", "No fire incidents found.")

    cursor.close()
    conn.close()

# Show average area burned by year
def show_average_area_by_year():
    conn = connect_db()
    if conn is None:
        return

    cursor = conn.cursor()
    query = """
    SELECT EXTRACT(YEAR FROM DataHoraAlerta) AS year, AVG(AreaTotal_ha) AS avg_area
    FROM RegistroIncendio
    GROUP BY year
    ORDER BY year;
    """
    cursor.execute(query)
    results = cursor.fetchall()

    if results:
        average_text = "\n".join(f"{int(result[0])}: {result[1]:.2f} ha" for result in results)
        messagebox.showinfo("Average Area Burned by Year", average_text)
    else:
        messagebox.showinfo("No Data", "No data available for average area burned by year.")

    cursor.close()
    conn.close()

# Create the main window
root = tk.Tk()
root.title("Fire Incident Data Analysis")
root.geometry("400x400")

# Create buttons for each feature
button_search_incident = tk.Button(root, text="Search Fire Incident by Locality", command=search_fire_incident)
button_search_firefighter = tk.Button(root, text="Search Firefighter by ID or Name", command=search_firefighter)
button_add_update_vehicle = tk.Button(root, text="Add/Update Vehicle Information", command=add_update_vehicle)
button_top_incidents = tk.Button(root, text="Show Top Fire Incidents by Area", command=show_top_incidents_by_area)
button_average_area = tk.Button(root, text="Show Average Area by Year", command=show_average_area_by_year)
button_exit = tk.Button(root, text="Exit", command=root.quit)

# Place buttons on the window
button_search_incident.pack(pady=10)
button_search_firefighter.pack(pady=10)
button_add_update_vehicle.pack(pady=10)
button_top_incidents.pack(pady=10)
button_average_area.pack(pady=10)
button_exit.pack(pady=10)

# Run the GUI event loop
root.mainloop()
